## Given Code

In [ ]:
# %% Minimal setup
# If needed (uncomment in a notebook):
# !pip install requests python-dotenv

import os, json, textwrap, re, time
import requests

API_KEY  = os.getenv("OPENAI_API_KEY", "cse476")
API_BASE = os.getenv("API_BASE", "http://10.4.58.53:41701/v1")  
MODEL    = os.getenv("MODEL_NAME", "bens_model")              

def call_model_chat_completions(prompt: str,
                                system: str = "You are a helpful assistant. Reply with only the final answer—no explanation.",
                                model: str = MODEL,
                                temperature: float = 0.0,
                                timeout: int = 60) -> dict:
    """
    Calls an OpenAI-style /v1/chat/completions endpoint and returns:
    { 'ok': bool, 'text': str or None, 'raw': dict or None, 'status': int, 'error': str or None, 'headers': dict }
    """
    url = f"{API_BASE}/chat/completions"
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type":  "application/json",
    }
    payload = {
        "model": model,
        "messages": [
            {"role": "system", "content": system},
            {"role": "user",   "content": prompt}
        ],
        "temperature": temperature,
        "max_tokens": 128,
    }

    try:
        resp = requests.post(url, headers=headers, json=payload, timeout=timeout)
        status = resp.status_code
        hdrs   = dict(resp.headers)
        if status == 200:
            data = resp.json()
            text = data.get("choices", [{}])[0].get("message", {}).get("content", "")
            return {"ok": True, "text": text, "raw": data, "status": status, "error": None, "headers": hdrs}
        else:
            # try best-effort to surface error text
            err_text = None
            try:
                err_text = resp.json()
            except Exception:
                err_text = resp.text
            return {"ok": False, "text": None, "raw": None, "status": status, "error": str(err_text), "headers": hdrs}
    except requests.RequestException as e:
        return {"ok": False, "text": None, "raw": None, "status": -1, "error": str(e), "headers": {}}


## Prompts

In [ ]:
SYSTEM_COT = """ You are a helpful reasoning assistant. 
Think step by step to solve the problem, but ONLY output the final answer.
When you are finished, output exactly one line in this format: Final answer : <answer>"""

## Helper Funcitons

In [ ]:
# similar to the parse_action function from mini_lab 5
# look at that lab for reference

def get_final_answer(text: str):
    """ 
    Get the final answer after 'Final answer:'.
    If nothing is found, return the whole text stripped or blank
    """

    if not text:
        return ""
    
    # just to get the final answer into a nice format
    lower_case = text.lower()
    ans = "final answer:"
    i = lower_case.find(ans)
    if i == -1:
        return text.strip()
    
    return text[ i + len(ans):].strip()



## Function Implementation 

### Actual Agent

In [ ]:
def run_agent(question):

    # first step: do CoT
    # second step: do RAP (this refines the CoT answer)
    # third step: do self consistency (small k; maybe 5-7)

    # return final answer

### Chain of thought (CoT)

In [ ]:
def run_cot(question: str, temperature: float = 0.0) -> str:
    user_prompt = f""" Problem: {question} 
Think step by step and output the answer with the following format: Final answer: <answer> """
    
    response = call_model_chat_completions(prompt = user_prompt, system = SYSTEM_COT, temperature = temperature)

    

### RAP

In [ ]:
 def run_rap():